In [1]:
import copy
import bokeh
from bokeh.plotting import show

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms


import flowkit as fk
import glob as glob

bokeh.io.output_notebook()
%matplotlib inline

_ = plt.ioff()

ModuleNotFoundError: No module named 'flowkit'

In [2]:
list = np.sort(glob.glob("/Users/calebmcgrath/Library/CloudStorage/OneDrive-ColoradoSchoolofMines/Research/Ram Data/Geometric mean calculations/Quenching DIP50H50_Cell Uptake_12/*.fcs"))

In [3]:
fcs_path = list[31]

In [4]:
sample = fk.Sample(fcs_path)

In [5]:
sample

Sample(v3.1, E2 NP-5_002.fcs, 51 channels, 8792 events)

In [6]:
df_events = sample.as_dataframe(source='raw')

In [7]:
f = sample.plot_scatter('FSC-A', 'SSC-A', source='raw')
show(f)

In [8]:
SSC_idx = sample.get_channel_index('SSC-A')
FSC_idx = sample.get_channel_index('FSC-A')
events_SSC= sample.get_channel_events(SSC_idx,source = 'raw')
events_FSC= sample.get_channel_events(FSC_idx,source = 'raw')

In [9]:
g_strat = fk.GatingStrategy()

In [10]:
dim_SSC = fk.Dimension('SSC-A')
dim_FSC = fk.Dimension('FSC-A')

In [11]:
vertices = [
    (1200000,400000),
    (2400000,500000),
    (2800000,1000000),
    (3000000,2000000),
    (2600000,2500000),
    (1800000,2800000),
    (1200000,2800000),
    (670000,2200000),
    (470000,1800000),
    (505000,1000000)
]

poly_gate = fk.gates.PolygonGate(
    'Cells', 
    dimensions=[dim_FSC, dim_SSC],
    vertices=vertices
)

In [12]:
g_strat.add_gate(poly_gate, gate_path=('root',))
res = g_strat.gate_sample(sample)

In [13]:
res.report

,sample,gate_path,gate_name,gate_type,quadrant_parent,parent,count,absolute_percent,relative_percent,level
0,E2 NP-5_002.fcs,"(root,)",Cells,PolygonGate,None,root,2874,32.688808,32.688808,1


In [14]:
sample_gate = g_strat.gate_sample(sample)
sample_cells_gated = sample_gate.get_gate_membership('Cells')

In [15]:
p = sample.plot_scatter(
    'FSC-A', 'SSC-A', source='raw', subsample=False, color_density = True, highlight_indices=sample_cells_gated)
show(p)

In [16]:
gs_results = g_strat.gate_sample(sample, verbose=True)

cell_gate_membership = gs_results.get_gate_membership('Cells')
cell_gate_membership.sum()

E2 NP-5_002.fcs: processing gate Cells


2874

In [17]:
def gated_cells (fcs_df, cell_gate_membership_array):
    """This function allows takes in a binary array for cell membership to a gate, along
    with the original dataframe from all the gates cells, and outputs a datafram that only
    include the values relavent to the cells containted within that gate"""

    #intializes column title array as well as the array to store filtered values
    titles = fcs_df.columns.values
    array_for_df = []

    #loops through titles filtering for cells that are contained within the gate
    for i in titles:
        array = df_events[i]
        new_array = []
        for i in range(len(array)):
            if cell_gate_membership_array[i]:
                new_array.append(array[i])

        array_for_df.append(new_array)

    #intializing dictionary to create data frame from
    d = {}

    #matching data with titles in dictionary for dataframe creation
    for i in range(len(titles)):
        d[titles[i]]= array_for_df[i]

    #creating dataframe
    df = pd.DataFrame(d)
    
    return df

In [18]:
cells_df = gated_cells(df_events, cell_gate_membership)
cells_df

,Time,FSC-W,FSC-H,FSC-A,SSC-W,SSC-H,SSC-A,B1-H,B1-A,B2-H,...,R4-H,R4-A,R5-H,R5-A,R6-H,R6-A,R7-H,R7-A,R8-H,R8-A
,,,,,,,,,,,...,,,,,,,,,,
0,0.0000,9.926173e+05,1288593.0,2.131800e+06,1.365004e+06,903635.0,2055774.875,69.599998,37.228127,68.500000,...,47744.0,83695.234375,36014.0,64417.410156,23032.0,40412.214844,17931.0,32002.792969,8438.0,14837.812500
1,0.0269,9.912548e+05,844016.0,1.394392e+06,1.269633e+06,721902.0,1527584.000,633.200012,1167.546875,624.700012,...,13993.0,22069.187500,10121.0,17451.832031,6484.0,11438.973633,5404.0,8559.048828,2154.0,3692.887451
2,0.0311,1.031703e+06,960919.0,1.652305e+06,1.362618e+06,862849.0,1959555.500,6415.899902,13757.872070,7082.500000,...,26585.0,61463.242188,20369.0,48020.289062,13114.0,30703.619141,9972.0,23518.841797,4880.0,11219.951172
3,0.0468,1.026454e+06,768596.0,1.314880e+06,1.034506e+06,431133.0,743349.500,58.299999,-58.368752,91.400002,...,62416.0,88212.484375,46162.0,68238.617188,30507.0,44217.605469,24282.0,34043.890625,10710.0,15955.653320
4,0.0772,9.200721e+05,956349.0,1.466517e+06,1.102465e+06,1032949.0,1897984.125,73.400002,72.414383,120.700005,...,41316.0,68222.796875,31286.0,53599.238281,20705.0,34454.933594,15660.0,26088.257812,7448.0,12633.336914
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2869,190.7658,1.094109e+06,293934.0,5.359929e+05,1.533653e+06,512764.0,1310670.375,83.200005,64.349998,94.500000,...,15783.0,34238.972656,12242.0,26164.140625,7425.0,16661.099609,5740.0,12688.057617,2715.0,5540.613770
2870,190.7803,6.822236e+05,826551.0,9.398211e+05,1.113950e+06,958633.0,1779781.125,126.400002,49.658127,75.900002,...,9082.0,14770.410156,7035.0,11988.524414,4362.0,7387.342285,3407.0,5488.031250,1681.0,2496.386230
2871,191.4368,9.586182e+05,547843.0,8.752871e+05,1.376304e+06,995464.0,2283436.000,97.099998,60.238750,100.500000,...,21754.0,44036.562500,16549.0,34495.542969,10324.0,21672.539062,8072.0,16791.558594,4247.0,7675.263672


In [19]:
sample = fk.Sample(cells_df, sample_id='Cells Gate')
f = sample.plot_scatter('SSC-A', 'SSC-H', source='raw')
show(f)

In [20]:
dim_SSC_A = fk.Dimension('SSC-A')
dim_SSC_H = fk.Dimension('SSC-H')


In [21]:
center = (1500000, 700000)
cov = [[9.74944434e+11, 4.92652573e+11], [4.92652573e+11, 2.82662600e+11]]
dist = 1000000

ellipse_gate = fk.gates.EllipsoidGate(
    'Singlet', 
    dimensions=[dim_SSC_A, dim_SSC_H], 
    coordinates=center,
    covariance_matrix=cov,
    distance_square=dist
)

In [22]:
g_strat.add_gate(ellipse_gate, gate_path=('root','Cells'))
res = g_strat.gate_sample(sample)

In [23]:
res.report

,sample,gate_path,gate_name,gate_type,quadrant_parent,parent,count,absolute_percent,relative_percent,level
0,Cells Gate,"(root,)",Cells,PolygonGate,None,root,2874,100.0,100.0,1
1,Cells Gate,"(root, Cells)",Singlet,EllipsoidGate,None,Cells,2874,100.0,100.0,2


In [24]:
sample_gate = g_strat.gate_sample(sample)
sample_cells_gated = sample_gate.get_gate_membership('Singlet')

In [25]:
p = sample.plot_scatter(
    'SSC-A', 'SSC-H', source='raw', subsample=False, color_density = True, highlight_indices=sample_cells_gated)
show(p)

In [26]:
quad_div1 = fk.QuadrantDivider(
    'Cellular Uptake',
    'R3-A',
    compensation_ref='uncompensated', 
    transformation_ref=None, 
    values=[2500]
)

quad_divs = [quad_div1]

# the 2 dividers above will be used to divide the space into 4 quadrants
quad_1 = fk.gates.Quadrant(
    quadrant_id='Cy5 positive',
    divider_refs=['Cellular Uptake'],
    divider_ranges=[(3000, None)]
)
quad_3 = fk.gates.Quadrant(
    quadrant_id='Cy5 negative',
    divider_refs=['Cellular Uptake'],
    divider_ranges=[(None, 3000)]
)

quadrants = [quad_1, quad_3]

# We can now construct our QuadrantGate
quad_gate1 = fk.gates.QuadrantGate(
    'Viability Gate', 
    dividers=quad_divs, 
    quadrants=quadrants
)

In [27]:
g_strat.add_gate(quad_gate1, gate_path=('root','Cells','Singlet',))
res = g_strat.gate_sample(sample)

In [28]:
res.report

,sample,gate_path,gate_name,gate_type,quadrant_parent,parent,count,absolute_percent,relative_percent,level
0,Cells Gate,"(root,)",Cells,PolygonGate,None,root,2874,100.000000,100.000000,1
1,Cells Gate,"(root, Cells)",Singlet,EllipsoidGate,None,Cells,2874,100.000000,100.000000,2
3,Cells Gate,"(root, Cells, Singlet)",Cy5 negative,QuadrantGate,Viability Gate,Singlet,113,3.931802,3.931802,3
2,Cells Gate,"(root, Cells, Singlet)",Cy5 positive,QuadrantGate,Viability Gate,Singlet,2761,96.068198,96.068198,3


In [30]:
sample_gate = g_strat.gate_sample(sample)
sample_cells_gated = sample_gate.get_gate_membership('Cy5 positive')

In [33]:
p = sample.plot_scatter(
    'R2-A', 'B2-A', source='raw', subsample=False, color_density = True, highlight_indices=sample_cells_gated)
show(p)

In [36]:
quad_div2 = fk.QuadrantDivider(
    'Transfection',
    'B2-A',
    compensation_ref='uncompensated', 
    transformation_ref=None, 
    values=[2500]
)

quad_divs = [quad_div2]

# the 2 dividers above will be used to divide the space into 4 quadrants
quad_2 = fk.gates.Quadrant(
    quadrant_id='GFP Positive Cells',
    divider_refs=['Transfection'],
    divider_ranges=[(10000, None)]
)
quad_4 = fk.gates.Quadrant(
    quadrant_id='GFP Negative Cells',
    divider_refs=['Transfection'],
    divider_ranges=[(None, 10000)]
)

quadrants = [quad_2, quad_4]

# We can now construct our QuadrantGate
quad_gate2 = fk.gates.QuadrantGate(
    'Transfection Gate', 
    dividers=quad_divs, 
    quadrants=quadrants
)

In [37]:
g_strat.add_gate(quad_gate2, gate_path=('root','Cells','Singlet','Viability Gate','Cy5 positive',))
res = g_strat.gate_sample(sample)

In [38]:
res.report

,sample,gate_path,gate_name,gate_type,quadrant_parent,parent,count,absolute_percent,relative_percent,level
0,Cells Gate,"(root,)",Cells,PolygonGate,None,root,2874,100.000000,100.000000,1
1,Cells Gate,"(root, Cells)",Singlet,EllipsoidGate,None,Cells,2874,100.000000,100.000000,2
3,Cells Gate,"(root, Cells, Singlet)",Cy5 negative,QuadrantGate,Viability Gate,Singlet,113,3.931802,3.931802,3
2,Cells Gate,"(root, Cells, Singlet)",Cy5 positive,QuadrantGate,Viability Gate,Singlet,2761,96.068198,96.068198,3
5,Cells Gate,"(root, Cells, Singlet, Viability Gate, Cy5 pos...",GFP Negative Cells,QuadrantGate,Transfection Gate,Cy5 positive,2736,95.198330,99.094531,5
4,Cells Gate,"(root, Cells, Singlet, Viability Gate, Cy5 pos...",GFP Positive Cells,QuadrantGate,Transfection Gate,Cy5 positive,25,0.869868,0.905469,5


In [39]:
text = g_strat.get_gate_hierarchy(output='ascii')

In [40]:
print(text)

root
╰── Cells
    ╰── Singlet
        ╰── Viability Gate
            ├── Cy5 positive
            │   ╰── Transfection Gate
            │       ├── GFP Positive Cells
            │       ╰── GFP Negative Cells
            ╰── Cy5 negative


In [41]:
sample_gate = g_strat.gate_sample(sample)
sample_cells_gated = sample_gate.get_gate_membership('GFP Positive Cells')

In [42]:
p = sample.plot_scatter(
    'B2-A', 'SSC-A', source='raw', subsample=False, color_density = True, highlight_indices=sample_cells_gated)
show(p)